## Agentic Sales Pipeline SambaNova & CrewAI


## Initial Imports

Get your Sambanova API Key Here https://cloud.sambanova.ai/


Get your Serper API Key Here https://serper.dev/ 

In [1]:
# Install Requirements

!pip install -r requirements.txt


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os

# Set API keys and base URL if not already configured in environment
if "SAMBANOVA_API_KEY" not in os.environ:
    os.environ["SAMBANOVA_API_KEY"] = "INSERT_SAMBANOVA_API_KEY_HERE"
    
if "SAMBANOVA_BASE_URL" not in os.environ:
    os.environ["SAMBANOVA_BASE_URL"] = "https://api.sambanova.ai/v1"
    
if "SERPER_API_KEY" not in os.environ:
    os.environ["SERPER_API_KEY"] = "INSERT_SERPER_API_KEY_HERE"


In [3]:
from crewai import LLM
import os
# # # Set Up Sambanova LLM

from utils import get_sambanova_api_key, update_task_output_format
llm = LLM(model="sambanova/Meta-Llama-3.1-70B-Instruct", api_key=get_sambanova_api_key(), base_url=os.getenv("SAMBANOVA_BASE_URL"))
manager_llm = LLM(model="sambanova/Meta-Llama-3.1-70B-Instruct", api_key=get_sambanova_api_key(), base_url=os.getenv("SAMBANOVA_BASE_URL"))

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

## Loading Tasks and Agents YAML files

In [5]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [6]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

In [7]:

from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

parser = PydanticOutputParser(pydantic_object=LeadScoringResult)  
format_instructions = parser.get_format_instructions()

tasks_config = update_task_output_format(lead_tasks_config, 'lead_scoring_and_validation', format_instructions)

## Importing Tools

In [8]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [9]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  llm=llm
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [10]:
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist'],
  llm=llm
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist'],
  llm=llm
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True,
)

2024-12-09 15:27:20,433 - 8560594752 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [11]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "Kwasi Ankomah",
                    "job_title": "Architect",
                    "company": "SambaNova",
                    "email": "kwasi@samaba.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [12]:
flow.plot()

Plot saved as crewai_flow.html


In [13]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [14]:
import nest_asyncio
import asyncio

nest_asyncio.apply()

async def run_flow():
    return flow.kickoff()

emails = asyncio.run(run_flow())

2024-12-09 15:27:20,468 - 8560594752 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int

2024-12-09 15:27:39,005 - 8560594752 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Scorer and Validator
## Final Answer: 
{
    "personal_info": {
        "name": "Kwasi Ankomah",
        "job_title": "Architect",
        "role_relevance": 8,
        "professional_background": "Kwasi Ankomah is an architect at SambaNova, a company that specializes in AI and machine learning solutions."
    },
    "company_info": {
        "company_name": "SambaNova",
        "industry": "Artificial Intelligence and Machine Learning",
        "company_size": 350,
        "revenue": 122200000,
        "market_presence": 8
    },
    "lead_score": {
        "score": 85,
        "scoring_criteria": [
            "Role Relevance: 8/10",
            "Company Size: 7/10",
            "Market Presence: 8/10",
            "Cultural Fit: 8/10"
        ],
        "validation_notes": "The lead's use case, 'Using AI Agent to do better data enrichment,' aligns with CrewAI's product and ICP, making it a potential fit for the company's solution."
    }
}


# Agent: Email Content Writ

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [15]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0115


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,76661,71847,4814,24


In [16]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,2301,1606,695,3


## Inspecting Results

In [17]:
scores = flow.state["score_crews_results"]

In [18]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,Kwasi Ankomah
Job Title,Architect
Role Relevance,8
Professional Background,"Kwasi Ankomah is an architect at SambaNova, a company that specializes in AI and machine learning solutions."
Company Name,SambaNova
Industry,Artificial Intelligence and Machine Learning
Company Size,350
Revenue,122200000.000000
Market Presence,8
Lead Score,85


## Results

In [19]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

Hi Kwasi Ankomah,  As an Architect at SambaNova, a leading company in AI and
machine learning solutions, I believe you understand the importance of efficient
automation in driving business growth. Your interest in using AI Agents for
better data enrichment resonates with our mission at CrewAI.  Our Multi-Agent
Orchestration Platform is designed to help enterprise companies like SambaNova
streamline their automation processes across various verticals. By leveraging
our platform, you can unlock the full potential of AI Agents and achieve
seamless data enrichment.  I'd love to explore how CrewAI can address your
specific needs and enhance SambaNova's AI and machine learning capabilities.
Take the next step and schedule a meeting with me to discuss how our solution
can benefit your team.  **Ready to Unlock the Power of AI Agents? Schedule a
Meeting Now**  Click this link to schedule a meeting at your convenience:
[Insert Calendly link]  Alternatively, reply to this email with your
availabi

## How Complex Can it Get?

In [20]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):
    
  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "Kwasi Ankomah",
          "job_title": "Architect",
          "company": "Sambanova",
          "email": "kwasi@sambanova.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads)
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [21]:
flow = SalesPipeline()
flow.plot()

Plot saved as crewai_flow.html


In [22]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)